### ----------------------------------------------------------------------------------------------------------
## 社会変革型 医療データサイエンティスト育成講座
# Chapter 3: モデル選択
### ----------------------------------------------------------------------------------------------------------

#### 1. データの準備・前処理

In [ ]:
# データのロード
import pandas as pd
bace_data = pd.read_csv("~/bace_data.csv")
bace_data.head()

In [ ]:
import numpy as np

# 今回は説明変数をすべて使用します！
X = np.array(bace_data.iloc[:,1:-2])
y = np.array(bace_data[['pIC50']])

In [ ]:
# 頻用される関数である"train_test_split"を用いて、データを訓練用と評価用に分割します
# test_size=1/4とすることで、train:test = 3:1 となります。もちろん比は変更可能です

# データの中からランダムにtrain用とtest用をピックアップしてくれるのですが、セルを実行するたびに
# データが毎回変わってしまうと不便な場合もあります。そこで"random_state=1"とすることで、
# 毎回同じ分割をするよう指定することができます。数字を変えることで異なる分割が得られます

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=1/4, 
                                                    random_state=1)

# data shapeを確認しておきます。正しく分割されていることがわかります
print(X_train.shape)
print(X_test.shape)

In [ ]:
# こちらも頻用される関数である"StandardScaler"を用いて、データの標準化を行います
from sklearn.preprocessing import StandardScaler

# インスタンスを作成します
scaler = StandardScaler()

# train dataでscalerを作成し、同じscalerをtestに対しても適用します
X_train_st = scaler.fit_transform(X_train)
X_test_st = scaler.transform(X_test)

#### 2. 交差検証

In [ ]:
# sklearnのcross_val_scoreを使用してモデルの評価を行います
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

# インスタンスを作成します
model = LinearRegression()

# modelとデータををcross_val_scoreに投入するだけで交差検証を行ってくれます
# cv=5は5-fold cross-validationを行うという意味です。もちろん他の数字も指定可能です
# scoringでは、評価指標を指定できます
cv_score = cross_val_score(model, X_train_st, y_train, cv=5, scoring='neg_mean_absolute_error')

# 最後に計算されたスコアを表示します
print('cv scores: ', cv_score)
print('mean score: ', cv_score.mean())

In [ ]:
# 使用可能な評価関数一覧
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

#### 4. 変数選択

In [ ]:
# 正則化なしの線形回帰モデルによるfitting
from sklearn.linear_model import LinearRegression
linModel = LinearRegression()
linModel.fit(X_train_st,y_train)

In [ ]:
# 評価用データのX_testからy_testを予測
y_pred = linModel.predict(X_test_st)

In [ ]:
# 評価用データへの当てはまりの指標としてMSEを計算
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

In [ ]:
# Lassoによるfitting
from sklearn.linear_model import Lasso
lassoModel = Lasso()
lassoModel.fit(X_train_st,y_train)

In [ ]:
# Lassoの予測精度を計算
y_pred_lasso = lassoModel.predict(X_test_st)
mean_squared_error(y_test, y_pred_lasso)

In [ ]:
# Ridgeによるfitting
from sklearn.linear_model import Ridge
ridgeModel = Ridge()
ridgeModel.fit(X_train_st, y_train)

In [ ]:
# Ridgeの予測精度を計算
y_pred_ridge = ridgeModel.predict(X_test_st)
mean_squared_error(y_test, y_pred_ridge)

In [ ]:
# Ridgeで最適ハイパーパラメータ探索
from sklearn.linear_model import RidgeCV
ridgeModel_cv = RidgeCV()
ridgeModel_cv.fit(X_train_st,y_train)

In [ ]:
# RidgeCVの予測精度を計算
y_pred_ridgecv = ridgeModel_cv.predict(X_test_st)
mean_squared_error(y_test, y_pred_ridgecv)